# GBIF API


In [1]:
import requests
import json

def obtener_taxon_key(nombre_cientifico):
    url = 'https://api.gbif.org/v1/species/match'

    parametros = {
        'name': nombre_cientifico,
        'strict': True  # Asegura una coincidencia exacta
    }

    respuesta = requests.get(url, params=parametros)
    
    if respuesta.status_code == 200:
        datos = respuesta.json()
        
        # Se revisa si la llave usageKey está presente en datos
        if 'usageKey' in datos:
            return datos['usageKey']
        else:
            return None
    else:
        return None

# Nombre científico de la especie
nombre_cientifico = 'Panthera onca'

# taxonKey de la especie
taxon_key = obtener_taxon_key(nombre_cientifico)

if taxon_key:
    # Se encontró el taxonKey

    # URL
    url = 'https://api.gbif.org/v1/occurrence/search'

    # Parámetros iniciales de la solicitud GET
    limit = 300
    offset = 0

    parametros = {
        'country': 'CR',          # Código de país ISO para Costa Rica
        'taxonKey': taxon_key,
        'limit': limit,
        'offset': offset,
        'hasCoordinate': 'true'   # Solo se incluyen registros de presencia con coordenadas
    }

    features = []
    total_count = None  # Se obtendrá en la primera solicitud

    while True:
        # Solicitud GET
        respuesta = requests.get(url, params=parametros)

        # Se verifica si la solicitud fue exitosa
        if respuesta.status_code == 200:
            # Se cargan en formato JSON los datos retornados
            datos_registros = respuesta.json()

            if total_count is None:
                total_count = datos_registros['count']
                print(f"Número total de registros: {total_count}")

            # Se extraen las ocurrencias
            registros = datos_registros['results']

            if not registros:
                break  # No hay más registros

            # Se recorren los datos de registros de presencia y se convierten a features GeoJSON
            for registro in registros:
                especie = registro.get('species')
                fecha = registro.get('eventDate')
                localidad = registro.get('locality')
                latitud = registro.get('decimalLatitude')
                longitud = registro.get('decimalLongitude')

                print(f"Especie: {especie}")
                print(f"Fecha: {fecha}")
                print(f"Localidad: {localidad}")
                print(f"Longitud: {longitud}, Latitud: {latitud}")
                print("-" * 40)        

                # Se verifica que las coordenadas están presentes
                if latitud is not None and longitud is not None:
                    feature = {
                        "type": "Feature",
                        "geometry": {
                            "type": "Point",
                            "coordinates": [longitud, latitud]  # GeoJSON utiliza el orden [longitud, latitud]
                        },
                        "properties": {
                            "species": especie,
                            "eventDate": fecha,
                            "locality": localidad
                        }
                    }
                    features.append(feature)

            # Incrementar offset
            offset += limit
            parametros['offset'] = offset

            if offset >= total_count:
                break

        else:
            # Se produjo un error
            print(f"Error {respuesta.status_code}")
            break

    # Crear el FeatureCollection GeoJSON
    geojson = {
        "type": "FeatureCollection",
        "features": features
    }

    # Almacenamiento de datos en un archivo llamado "registros-presencia.geojson"
    with open('registros-presencia.geojson', 'w', encoding='utf-8') as archivo:
        json.dump(geojson, archivo, indent=4, ensure_ascii=False)

    print(f"Se han guardado {len(features)} registros de presencia en el archivo 'registros-presencia.geojson'")
else:
    print(f'No se encontró el taxonKey de {nombre_cientifico}')

Número total de registros: 123
Especie: Panthera onca
Fecha: 2024-01-30T15:18:56
Localidad: None
Longitud: -83.579157, Latitud: 10.467856
----------------------------------------
Especie: Panthera onca
Fecha: 2024-01-25T07:16
Localidad: None
Longitud: -84.655167, Latitud: 10.22839
----------------------------------------
Especie: Panthera onca
Fecha: 2024-02-01T13:24:14
Localidad: None
Longitud: -83.566878, Latitud: 10.449497
----------------------------------------
Especie: Panthera onca
Fecha: 2024-02-06T09:51
Localidad: None
Longitud: -83.565366, Latitud: 8.466594
----------------------------------------
Especie: Panthera onca
Fecha: 2024-02-08T16:38
Localidad: None
Longitud: -83.563344, Latitud: 10.454353
----------------------------------------
Especie: Panthera onca
Fecha: 2024-02-09T11:42
Localidad: None
Longitud: -83.548497, Latitud: 10.469183
----------------------------------------
Especie: Panthera onca
Fecha: 2024-02-09T11:45
Localidad: None
Longitud: -83.449484, Latitud: 1

In [2]:
import pandas as pd
import geopandas as gpd
import plotly.express as px

# Lista de nombres de meses en español
meses_espanol = [
    'Desconocido',  # Índice 0
    'Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
    'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre'
]

# Lectura del archivo GeoJSON en un GeoDataFrame
gdf = gpd.read_file('registros-presencia.geojson')

# Conversión de 'eventDate' a hilera de texto y luego a formato datetime
gdf['eventDate'] = gdf['eventDate'].astype(str)
gdf['eventDate'] = pd.to_datetime(gdf['eventDate'], errors='coerce')

# Extracción del número del mes
gdf['mes_num'] = gdf['eventDate'].dt.month
# Mapear los números de mes a nombres en español
gdf['mes'] = gdf['mes_num'].apply(
    lambda x: meses_espanol[int(x)] if pd.notnull(x) and x >= 1 and x <= 12 else 'Desconocido'
)

# Conteo y ordenamiento de la cantidad de registros de presencia por mes
registros_por_mes = gdf.groupby(['mes_num', 'mes']).size().reset_index(name='cantidad')
registros_por_mes = registros_por_mes.sort_values('mes_num')

# Creación del gráfico de barras interactivo con plotly
fig = px.bar(
    registros_por_mes,
    x='mes',
    y='cantidad',
    title='Cantidad de registros de presencia por mes',
    labels={'mes': 'Mes', 'cantidad': 'Cantidad de registros de presencia'},
    text='cantidad'
)

# Ajustes al gráfico
fig.update_layout(
    xaxis_tickangle=-45,
    xaxis_title='Mes',
    yaxis_title='Registros de presencia',
    title_x=0.5,
    template='plotly_white'
)

# Despliegue del gráfico
fig.show()

In [3]:
import leafmap

# Creación de un objeto Map de leafmap centrado en Costa Rica
m = leafmap.Map(center=(9.972725, -84.19451), zoom=7)

# Adición de datos GeoJSON al mapa
m.add_point_layer(
    'registros-presencia.geojson', 
    popup=["species", "eventDate", "locality"], 
    layer_name="Registros de presencia"
)

# Despliegue del mapa
m

Map(center=[9.972725, -84.19451], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …